In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Collect data since 2009
us = pd.read_csv('csis_wapo_domestic_terrorism.csv')

us.columns = us.columns.str.rstrip('_csis')
us = us.drop(['LE_statu','MIL_statu'], axis=1)
us = us[us.Year >= 2009]
us.head()

,Year,Month,Day,State,City,Orientation,vict_killed,Target,Weapon,Lat,Long,type_extremism_1,type_extremism_2,victim_category,victim_specific1,victim_specific2,susp_total,susp_hometown,susp_social_media_use
0,2021,1.0,15.0,California,Napa,Violent Far-right,0,"Government, Military, and Police",Explosives and Incendiaries,38.2975,-122.2869,anti government,anti left,group,government,left,1.0,"Napa, CA",various
1,2021,1.0,12.0,New York,New York - Queens,Violent Far-right,0,"Government, Military, and Police",Firearms,40.7310,-73.7960,white supremacist,anti government,group,government,NaN,1.0,"New York, NY",parler
2,2021,1.0,4.0,New York,New York - Queens,Violent Far-right,0,Business,Explosives and Incendiaries,40.7310,-73.7960,white supremacist,anti semitic,individual,BLM,NaN,1.0,"Lyne, MA",various
3,2021,1.0,25.0,Washington,Tacoma,Violent Far-left,0,"Government, Military, and Police",Explosives and Incendiaries,47.2530,-122.4440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,1.0,20.0,Washington,Seattle,Violent Far-left,0,Business,Melee,47.6060,-122.3330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Converge names for NYC and DC
us.City = us.City.replace(['New York - Queens','New York-Brooklyn','New York-Queens','New York'], 'New York City')
us.City = us.City.replace(['DC','D.C.','District of Columbia'], 'Washington D.C.')
us.State = us.State.replace(['District of Columbia'], 'Washington D.C.')

In [4]:
# Label Washington DC as both state and city
us.loc[us.State == 'Washington D.C.', 'City'] = 'Washington D.C.'

In [5]:
# Feature reduction
us_clean = us[['Year', 'State', 'City', 'Orientation', 'vict_killed', 'Target', 'Weapon', 'Lat', 'Long', 
               'type_extremism_1','type_extremism_2', 'victim_category', 'victim_specific1', 'susp_total',
               'susp_social_media_use']]
us_clean.head()

,Year,State,City,Orientation,vict_killed,Target,Weapon,Lat,Long,type_extremism_1,type_extremism_2,victim_category,victim_specific1,susp_total,susp_social_media_use
0,2021,California,Napa,Violent Far-right,0,"Government, Military, and Police",Explosives and Incendiaries,38.2975,-122.2869,anti government,anti left,group,government,1.0,various
1,2021,New York,New York City,Violent Far-right,0,"Government, Military, and Police",Firearms,40.7310,-73.7960,white supremacist,anti government,group,government,1.0,parler
2,2021,New York,New York City,Violent Far-right,0,Business,Explosives and Incendiaries,40.7310,-73.7960,white supremacist,anti semitic,individual,BLM,1.0,various
3,2021,Washington,Tacoma,Violent Far-left,0,"Government, Military, and Police",Explosives and Incendiaries,47.2530,-122.4440,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,Washington,Seattle,Violent Far-left,0,Business,Melee,47.6060,-122.3330,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# us_clean.to_csv('us.csv', index=False)

In [13]:
# Obtain most hit cities
most_attacked_cities = us_clean.City.value_counts()[:15]
most_attacked_cities

New York City      34
Washington D.C.    16
Portland           13
Seattle            10
Miami               7
Dallas              6
Chicago             6
Los Angeles         6
Tampa               5
Houston             5
Austin              5
Arlington           4
Spokane             4
Denver              4
Boston              4
Name: City, dtype: int64

In [14]:
# Get how many fatalities in these cities
killed_perCity = dict()
for city in most_attacked_cities.index:
    killed_perCity.update({city:sum(us_clean[us_clean.City == city]['vict_killed'])})
killed_perCity

{'New York City': 11,
 'Washington D.C.': 2,
 'Portland': 3,
 'Seattle': 1,
 'Miami': 0,
 'Dallas': 5,
 'Chicago': 0,
 'Los Angeles': 1,
 'Tampa': 0,
 'Houston': 0,
 'Austin': 1,
 'Arlington': 0,
 'Spokane': 0,
 'Denver': 1,
 'Boston': 5}

In [27]:
# Create dataframe of cities
us_cities = pd.DataFrame(us_clean.City.value_counts()[:15]).rename(columns={'City':'Instances'})

In [28]:
# Dataframe of attacks and deaths by most hit cities
top_cities = pd.DataFrame.from_dict(killed_perCity, orient='index', columns=['Kills']).join(us_cities, how='outer')
top_cities = top_cities.reset_index().rename(columns={'index':'City'})
top_cities

,City,Kills,Instances
0,New York City,11,34
1,Washington D.C.,2,16
2,Portland,3,13
3,Seattle,1,10
4,Miami,0,7
5,Dallas,5,6
6,Chicago,0,6
7,Los Angeles,1,6
8,Tampa,0,5
9,Houston,0,5


In [29]:
# Get most hit states and their number of fatalities
most_attacked_states = us_clean.State.value_counts()[:15]
killed_perState = dict()
for state in most_attacked_states.index:
    killed_perState.update({state:sum(us_clean[us_clean.State == state]['vict_killed'])})

killed_perState

{'California': 31,
 'New York': 12,
 'Florida': 63,
 'Texas': 41,
 'Washington': 1,
 'Oregon': 12,
 'Illinois': 1,
 'Georgia': 3,
 'Massachusetts': 7,
 'Washington D.C.': 2,
 'Pennsylvania': 15,
 'Missouri': 0,
 'Virginia': 1,
 'Ohio': 0,
 'North Carolina': 2}

In [30]:
# Dataframe of stats for most hit states
us_states = pd.DataFrame(us_clean.State.value_counts()[:15]).rename(columns={'State':'Instances'})
top_states = pd.DataFrame.from_dict(killed_perState, orient='index', columns=['Kills']).join(us_states, how='outer')
top_states = top_states.reset_index().rename(columns={'index':'State'})
top_states

,State,Kills,Instances
0,California,31,50
1,New York,12,49
2,Florida,63,40
3,Texas,41,38
4,Washington,1,29
5,Oregon,12,25
6,Illinois,1,18
7,Georgia,3,17
8,Massachusetts,7,17
9,Washington D.C.,2,16


In [31]:
top_cities.to_csv('top_15_cities.csv', index=False)
top_states.to_csv('top_15_states.csv', index=False)